In [1]:
import pandas as pd
import numpy as np

c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Alessandro\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Carica i dati
dati = pd.read_csv('bitcoin_data.csv')
dati

,Timestamp,Price
0,2013-04-28 00:00:00.000,135.300000
1,2013-04-29 00:00:00.000,141.960000
2,2013-04-30 00:00:00.000,135.300000
3,2013-05-01 00:00:00.000,117.000000
4,2013-05-02 00:00:00.000,103.430000
...,...,...
3875,2023-12-09 00:00:00.000,44202.176109
3876,2023-12-10 00:00:00.000,43745.491992
3877,2023-12-11 00:00:00.000,43757.959938
3878,2023-12-12 00:00:00.000,41200.962726


In [5]:
dati['Price'].rolling(window=14)

Rolling [window=14,center=False,axis=0,method=single]

In [ ]:
# Scelgo quante combinazioni dei parametri voglio
combinazioni = 1000

# Inizializza il dataframe dei risultati
risultati = pd.DataFrame()

# Funzione per calcolare l'obiettivo
def calcola_obiettivo(coefficiente, t):
    return coefficiente * t

# Funzione per calcolare la media mobile
def calcola_media_mobile(dati, giorni, tipo):
    if tipo == 'semplice':
        return dati.rolling(window=giorni).mean()
    elif tipo == 'esponenziale':
        return dati.ewm(span=giorni).mean()
    
def da_bitcoin_a_dollari(prezzo, bitcoin):
    return prezzo * bitcoin

def da_dollari_a_bitcoin(prezzo, dollari):
    return dollari / prezzo

# Simulazione
for _ in range(combinazioni):  # Numero di iterazioni della random search
    # Scegli i parametri casualmente
    obiettivo_variabile = np.random.choice([True, False])
    timeframe_variabile = np.random.choice([True, False])
    vendita = np.random.choice([True, False])
    tipo_media_breve = np.random.choice(['semplice', 'esponenziale'])
    tipo_media_lunga = np.random.choice(['semplice', 'esponenziale'])
    timeframe_VA = np.random.randint(1, 365)
    coefficiente_obiettivo = np.random.uniform(0.1, 10)
    investimento_base = np.random.uniform(0, 10000)
    timeframe_media_lunga = np.random.randint(2, 365)
    timeframe_media_breve = np.random.randint(1, timeframe_media_lunga)
    commissioni_vendita = np.random.uniform(0, 0.1)
    commissioni_acquisto = np.random.uniform(0, 0.1)
    acquisto_minimo = np.random.uniform(0, 1000)
    vendita_minima = np.random.uniform(0, 1000)

    # Calcola le medie mobili
    media_breve = calcola_media_mobile(dati['Price'], timeframe_media_breve, tipo_media_breve)
    media_lunga = calcola_media_mobile(dati['Price'], timeframe_media_lunga, tipo_media_lunga)

    # Operazioni del giorno 0
    if investimento_base >= acquisto_minimo:
        portafoglio = investimento_base
        bitcoin_posseduti = investimento_base / dati['Price'].iloc[0]
    else:
        portafoglio = 0
        bitcoin_posseduti = 0

    # Simula il trading
    for t in range(1, len(dati)):
        bitcoin_posseduti_in_dollari = da_bitcoin_a_dollari(dati['Price'].iloc[t], bitcoin_posseduti)
        prezzo_attuale = dati['Price'].iloc[t]

        # Calcolo il coefficiente obiettivo nel caso sia obiettivo_variabile = True
        if obiettivo_variabile:
            coefficiente_obiettivo = media_lunga.iloc[t]

        # Calcola l'obiettivo
        obiettivo = calcola_obiettivo(coefficiente_obiettivo, t)

        # Calcola la differenza tra il valore attuale e l'obiettivo
        differenza = obiettivo - bitcoin_posseduti_in_dollari


        # Se il valore attuale è inferiore all'obiettivo, compra
        if differenza >= 0 and differenza >= acquisto_minimo:
            portafoglio -= differenza * (1 + commissioni_acquisto)
            bitcoin_posseduti += da_dollari_a_bitcoin(prezzo_attuale, differenza)

        # Se il valore attuale è superiore all'obiettivo e la vendita è abilitata, vendi
        elif differenza < 0 and vendita and abs(differenza) >= vendita_minima:
            portafoglio += abs(differenza) * (1 - commissioni_vendita)
            bitcoin_posseduti -= da_bitcoin_a_dollari(prezzo_attuale, abs(differenza))

    # Salva i risultati
    risultati = risultati.append({
        'obiettivo_variabile': obiettivo_variabile,
        'timeframe_variabile': timeframe_variabile,
        'vendita': vendita,
        'tipo_media_breve': tipo_media_breve,
        'tipo_media_lunga': tipo_media_lunga,
        'timeframe_VA': timeframe_VA,
        'coefficiente_obiettivo': coefficiente_obiettivo,
        'investimento_base': investimento_base,
        'timeframe_media_lunga': timeframe_media_lunga,
        'timeframe_media_breve': timeframe_media_breve,
        'commissioni_vendita': commissioni_vendita,
        'commissioni_acquisto': commissioni_acquisto,
        'acquisto_minimo': acquisto_minimo,
        'vendita_minima': vendita_minima,
        'portafoglio_finale': portafoglio,
        'bitcoin_posseduti': bitcoin_posseduti,
        'obiettivo': obiettivo
    }, ignore_index=True)

print(risultati)

# Salva i risultati in un file CSV
# risultati.to_csv('Risultati strategia VA + medie mobili.csv', index=False)